<a href="https://colab.research.google.com/github/vanderbilt-data-science/job-coach-qa/blob/6-explore-prompts-ricky/prompt_explore_notebook_ricky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Job Search QA

## Problem Definition

- Job Coach Assistant for people in Autism Spectrum
- Make the UI as simple as possible (text/email or simple huggingface?)

## connecting Colab to Github

In [1]:
!pwd

/content


In [ ]:
!git clone https://github_token@github.com/vanderbilt-data-science/job-coach-qa.git

Cloning into 'job-coach-qa'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 5), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), 9.04 KiB | 1.51 MiB/s, done.


In [ ]:
%cd job-coach-qa

/content/job-coach-qa


In [ ]:
!git fetch origin
!git checkout '6-explore-prompts-ricky'

Branch '6-explore-prompts-ricky' set up to track remote branch '6-explore-prompts-ricky' from 'origin'.
Switched to a new branch '6-explore-prompts-ricky'


In [ ]:
!git branch

* 6-explore-prompts-ricky
  main


In [ ]:
!git pull

Already up to date.


## Commit Changes

In [ ]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"
!git commit -a -m "Your commit message"


In [ ]:
!git remote set-url origin https://<token>@github.com/username/repo.git
!git push

## Libraries

In [2]:
# install libraries here
# -q flag for "quiet" install
!pip install -q langchain
!pip install -q openai
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.9 MB/s eta 0:0

In [3]:
# import libraries here
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import openai
import os
import json
import pandas as pd
import numpy as np
import panel as pn
from getpass import getpass

## API Keys

Use these cells to load the API keys required for this notebook. The below code cell uses the `getpass` library.

In [8]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


## Model Setup

Use these cells to set up your model. The example code below initiates the OpenAI model using Langchain.

In [ ]:
llm = OpenAI()

## Data Loading

Information about the data/documents used for this notebook, along with explanations of code for importing data below.

In [5]:
# load data here
# Open the file and read its contents
with open('Anonymized Job Coach QA Test Doc (1).txt', 'r') as file:
    data = file.read()

print(data)  # print the content of the file


﻿Hello,


We are so excited for this semester’s partnership with Data Science Institute and Next Steps at Vanderbilt. Jonathan Wade will be interning Mondays and Wednesdays 2-5 pm and Fridays 12-4 pm starting Monday, January 31st. Jessica will be job coaching on Mondays and Wednesdays from 2-5 pm. It also used to be on Fridays from 2-4 pm but not anymore.


Below is important information and reminders:


Location: 1400 18th Ave S Building, Suite 2000, Nashville, TN 37212


Dress: Business casual attire (This includes items such as dress pants, khakis, polos and dress shirts)


Important Dates:


Vanderbilt University is off for Spring Break March 5th - March 13th. No internships will take place these days.


All internships end by Thursday, April 28th.


Important COVID-19 Information: Attached is a document outlining the COVID-19 guidelines all Vanderbilt University students must follow, including Next Steps interns and job coaches while at internship sites. Please note these may chan

In [ ]:
len(data)

5672

## Implementation

Add details about the solution implementation here, as well as any information relevant to the code below. Feel free to add more code/text for better organization.

### Test Summarization

In [9]:
# solution/ implementation here
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [12]:
text = data

prompt_1 = f"""
Behave like a professional job coach and perform the following action(s):
Summarize the following text.

The output format should be like
Section title 1: section content 1
Section title 2: section content 2
…
Section title N: section content N

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print(response)

Internship Schedule: Jonathan Wade interning Mondays and Wednesdays 2-5 pm, Fridays 12-4 pm starting January 31st. Jessica job coaching Mondays and Wednesdays 2-5 pm.

Location: 1400 18th Ave S Building, Suite 2000, Nashville, TN 37212

Dress: Business casual attire

Important Dates: Spring Break March 5th - March 13th (no internships), internships end by April 28th

COVID-19 Guidelines: Masks required, lunch breaks with physical distancing, symptom monitoring, and procedures for positive tests and close contacts

Career Development Resource Guide: Attached for student expectations and resources

Internship Coordinator: Lynda Tricia

Contacts: Intern (Jonathan Wade), Supervisor (Ruben Miller), Job Coach (Jessica Cho), Internship Coordinator (Lynda Tricia)

Next Steps at Vanderbilt - Safety Guidelines: Mask requirements, lunch break protocols, symptom monitoring, procedures for positive tests and close contacts, job coaching supports


### QA

In [20]:
def get_completion(prompt, model="gpt-3.5-turbo-16k"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [21]:
def collect_messages(_):
    if file_upload.value:
        file_contents = file_upload.value.decode()
        # Add the file contents to the context
        context.append({'role':'system', 'content': f"{file_contents}"})
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


#### GPT3.5-turbo

In [ ]:
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are AI job coach, an automated service to answer questions for new employees based on given job/employee manual. \
You first greet the user, then asks if the user wants to ask any question about the manual. \
You answer questions user asks iteratively until user does not have anything to ask. \
You ask one last time if the customer wants to ask anything else. \
Finally, return a table with columns 'Questions' and 'Answers' which record the questions user asks and answers you give. \
Remember, the users are people who are in Autism Spectrum, so make the answer simple enough. \
"""} ]  # accumulate messages

file_upload = pn.widgets.FileInput()
inp = pn.widgets.TextAreaInput(value="Hi", placeholder='Enter text here...', width=500, height=200)
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    file_upload,
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 297, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 115, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/protocol/messages/patch_doc.py", line 115, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.content, setter))
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_json(self, event_json, references, setter)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 259, in handle_json
 handler(doc, event_json, references, setter)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/events.py", line 300, in _handle_json
 cb(event_json["msg_data"])
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 355, in trigger_json_event
 model._trigger_event(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 119, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 247, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 115, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 413, in _comm_event
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 408, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 411, in _comm_event
 self._process_bokeh_event(doc, event)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 348, in _process_bokeh_event
 self._process_event(event)
 File "/usr/local/lib/python3.10/dist-packages/panel/widgets/button.py", line 183, in _process_event
 self.param.trigger('value')
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1993, in trigger
 self_.set_param(**dict(params, **triggers))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1929, in set_param
 return self_.update(kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 838, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 818, in eval
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "/usr/local/lib/python3.10/dist-packages/panel/depends.py", line 209, in wrapped
 return function(*combined_args, **combined_kwargs)
 File "<ipython-input-50-6c303fa06b5e>", line 9, in collect_messages
 response = get_completion_from_messages(context)
 File "<ipython-input-45-6e2e2a359290>", line 11, in get_completion_from_messages
 response = openai.ChatC

Column
    [0] FileInput()
    [1] TextAreaInput(height=200, placeholder='Enter text here...', sizing_mode='fixed', width=500)
    [2] Row
        [0] Button(name='Chat!')
    [3] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

#### GPT3.5-turbo-16k

In [23]:
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are an AI job coach, an automated service to answer questions for new employees based on given job/employee manual. \
You first greet the user, then ask if the user wants to ask any question about the manual. \
You answer questions user ask iteratively until user does not have anything to ask. \
You ask one last time if the customer wants to ask anything else. \
Finally, return a table with columns 'Questions' and 'Answers' which record the questions user asks and answers you give. \
Remember, the users are people who are in Autism Spectrum, so make the answer simple enough. \
"""} ]  # accumulate messages

file_upload = pn.widgets.FileInput()
inp = pn.widgets.TextAreaInput(value="Hi", placeholder='Enter text here...', width=500, height=200)
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    file_upload,
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] FileInput()
    [1] TextAreaInput(height=200, placeholder='Enter text here...', sizing_mode='fixed', width=500)
    [2] Row
        [0] Button(name='Chat!')
    [3] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

#### GPT4

In [ ]:
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are AI job coach, an automated service to answer questions for new employees based on given job/employee manual. \
You first greet the user, then asks if the user wants to ask any question about the manual. \
You answer questions user asks iteratively until user does not have anything to ask. \
You ask one last time if the customer wants to ask anything else. \
Finally, return a table with columns 'Questions' and 'Answers' which record the questions user asks and answers you give. \
"""} ]  # accumulate messages

file_upload = pn.widgets.FileInput()
inp = pn.widgets.TextAreaInput(value="Hi", placeholder='Enter text here...', width=500, height=200)
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    file_upload,
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] FileInput()
    [1] TextAreaInput(height=200, placeholder='Enter text here...', sizing_mode='fixed', width=500)
    [2] Row
        [0] Button(name='Chat!')
    [3] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

## Interface

Explain how the user will access the solution implemented above - through the notebook, Gradio, HuggingFace, etc. Include examples of using and testing the solution developed above.

In [ ]:
# code here





## Conclusion

Summarize the problem your solution addresses, and how the team accomplished this. Include any caveats or further considerations.